# This notebook creates a dataset for min/max winning scores
- min win df score
- max win df score
- median team score (real game scores)
- 75th percentile team score (real game scores)
- number of slate games
- median df score for each player position
- 75th percentile df score for each player position
- median df score of top 50% of players for each position over the previous W weeks
- 75th percentile df score of top 50% for each position over the previous W weeks

In [37]:
# Load all daily fantasy contest data
from datetime import date
import re
from typing import Optional

import pandas as pd

from fantasy_py import ContestStyle
from fantasy_py.lineup.strategy import GeneralPrizePool, FiftyFifty

MIN_DATE: date   # contests on or after this date
MAX_DATE: date   # contests BEFORE (exclusive of) this date
    
# days to use to identify top players going into a slate
TOP_PLAYER_DAYS = 21
# players above this percentil over the last TOP_PLAYER_DAYS are considered top players
TOP_PLAYER_PERCENTILE = .70

# positions to ignore, default is to use everything
COST_POS_DROP = set()
# rename positions in cost data
COST_POS_RENAME = {}

# SPORT = 'nfl'
# MIN_DATE = date(2020, 8, 1)
# MAX_DATE = date(2021, 4, 1)

SPORT = 'mlb'
MIN_DATE = date(2019, 1, 1)
MAX_DATE = date(2021, 1, 1)
DB_FILENAME = f"/home/delano/working/fantasy/mlb_hist_20082020.scored.db"
COST_POS_DROP = {'DH', 'RP'}
COST_POS_RENAME = {'SP': 'P'}

# SPORT = 'nba'
# MIN_DATE = date(2019, 8, 1)
# MAX_DATE = date(2020, 8, 1)

# SPORT = 'nhl'
# MIN_DATE = date(2020, 8, 1)
# MAX_DATE = date(2021, 4, 1)

# fanduel/draftkings/yahoo
SERVICE = 'draftkings'
STYLE: Optional[ContestStyle] = ContestStyle.CLASSIC
# GeneralPrizePool/FiftyFifty
CONTEST_TYPE = GeneralPrizePool

def infer_contest_style(title) -> ContestStyle:
    if SERVICE == 'draftkings':
        if ('Showdown' in title or
            re.match('.*.{2,3} vs .{2,3}\)', title)):
           return ContestStyle.SHOWDOWN
        return ContestStyle.CLASSIC
    if SERVICE == 'fanduel':
        return ContestStyle.SHOWDOWN if '@' in title else ContestStyle.CLASSIC
    if SERVICE == 'yahoo':
        if (' Cup ' in title or 
            ' to 1st]' in title or 
            ' 50/50' in title or
            'QuickMatch vs ' in title or 
            'H2H vs ' in title or
            '-Team' in title or   # N-team contests are classic
            'Freeroll' in title or
            'Quadruple Up' in title or
            title.endswith('Guaranteed [No Management Fee]')):
           return ContestStyle.CLASSIC
    raise NotImplementedError(f"Could not infer contest style for {SERVICE=} {title=}")    
    
def infer_contest_type(title) -> str:
    if SERVICE == 'draftkings':
        if re.match('.* vs\. [^)]+$', title):
            return 'H2H'
        return FiftyFifty.NAME if 'Double Up' in title else GeneralPrizePool.NAME
    if SERVICE == 'fanduel':
        if 'Head-to-head' in title:
            return 'H2H'
        return FiftyFifty.NAME if title.startswith('50/50') else GeneralPrizePool.NAME
    if SERVICE == 'yahoo':
        if (' QuickMatch vs ' in title or 
            'H2H vs ' in title):
            return 'H2H'
        if ' 50/50' in title:
            return FiftyFifty.NAME
        if (' Cup ' in title or 
            ' to 1st]' in title or 
            'Freeroll' in title or
            'Quadruple Up' in title or
            title.endswith('-Team') or                        # multi-team games are GPP if not caught by 50/50
            title.endswith('Team Winner Takes All') or        # treat winner takes all like a gpp
            title.endswith('Guaranteed [No Management Fee]')):
           return GeneralPrizePool.NAME
    raise NotImplementedError(f"Could not infer contest type for {SERVICE=} {title=}")
    

contest_df = pd.read_csv(SERVICE + ".contest.csv", parse_dates=['date']) \
               .query('sport == @SPORT and @MIN_DATE <= date < @MAX_DATE') \
               [['contest_id', 'date', 'title', 'top_score', 'last_winning_score']]
contest_df.date = contest_df.date.dt.normalize()

# add style and type
contest_df['style'] = contest_df.title.map(infer_contest_style)
contest_df['type'] = contest_df.title.map(infer_contest_type)
queries = []
if STYLE is not None:
    print(f"Filtering for {STYLE=}")
    queries.append('style == @STYLE')
if CONTEST_TYPE is not None:
    print(f"Filtering for {CONTEST_TYPE=}")
    queries.append('type == @CONTEST_TYPE.NAME')
if len(queries) > 0:
    contest_df = contest_df.query(' and '.join(queries))

# with pd.option_context('max_rows', 1000, 'max_colwidth', 100):
display(contest_df.sort_values(['style', 'type']))

Filtering for STYLE=<ContestStyle.CLASSIC: 'classic'>
Filtering for CONTEST_TYPE=<class 'fantasy_py.lineup.strategy.bet_lineup.GeneralPrizePool'>


,contest_id,date,title,top_score,last_winning_score,style,type
103,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP
145,91766081,2020-09-12,MLB $300 Dime Time [Just $0.10!] (9/10),200.30,149.7,classic,GPP
146,91766081,2020-09-12,MLB $300 Dime Time [Just $0.10!] (8/10),200.30,149.7,classic,GPP
147,91766081,2020-09-12,MLB $300 Dime Time [Just $0.10!] (7/10),200.30,149.7,classic,GPP
148,91766081,2020-09-12,MLB $300 Dime Time [Just $0.10!] (6/10),200.30,149.7,classic,GPP
...,...,...,...,...,...,...,...
1725,71360839,2019-04-11,MLB $100K Player Appreciation Contest,184.95,163.1,classic,GPP
1730,71277566,2019-04-10,MLB $5K Quarter Jukebox [Just $0.25!] (2/2),237.95,192.6,classic,GPP
1731,71277566,2019-04-10,MLB $5K Quarter Jukebox [Just $0.25!] (1/2),237.95,192.6,classic,GPP
1733,71238529,2019-04-09,MLB $7.5K Quarter Jukebox [Just $0.25!] (2/2),225.75,151.1,classic,GPP


In [38]:
draft_df = pd.read_csv(SERVICE + ".draft.csv", parse_dates=['date']) \
             .query('sport == @SPORT and @MIN_DATE <= date < @MAX_DATE')
display(draft_df)
assert len(draft_df) > 0, "no draft data found"

draft_df['service'] = draft_df.contest.map(lambda contest: contest.split('-', 1)[0])
if SERVICE == 'fanduel':
    SERVICE_ABBR = 'fd'
elif SERVICE == 'draftkings':
    SERVICE_ABBR = 'dk'
elif SERVICE == 'yahoo':
    SERVICE_ABBR = 'y'
else:
    raise NotImplementedError()
draft_df = draft_df.query('service == @SERVICE_ABBR and team_abbr.notnull()') \
    [['position', 'name', 'team_abbr', 'contest_id']]
display(draft_df)

,position,name,team_abbr,draft_pct,contest,date,sport,contest_id
2272,CPT,T. Gonsolin,LAD,6.7,dk-mlb-20201021-MLB Showdown $2.5K Quarter Juk...,2020-10-21 20:08:00,mlb,94886275
2273,NaN,T. Gonsolin,LAD,19.9,dk-mlb-20201021-MLB Showdown $2.5K Quarter Juk...,2020-10-21 20:08:00,mlb,94886275
2274,CPT,B. Snell,TB,16.3,dk-mlb-20201021-MLB Showdown $2.5K Quarter Juk...,2020-10-21 20:08:00,mlb,94886275
2275,NaN,B. Snell,TB,39.9,dk-mlb-20201021-MLB Showdown $2.5K Quarter Juk...,2020-10-21 20:08:00,mlb,94886275
2276,CPT,J. Turner,LAD,5.4,dk-mlb-20201021-MLB Showdown $2.5K Quarter Juk...,2020-10-21 20:08:00,mlb,94886275
...,...,...,...,...,...,...,...,...
55808,2B,T. La Stella,LAA,5.2,dk-mlb-20190409-MLB $7.5K Quarter Jukebox [Jus...,2019-04-09 19:05:00,mlb,71238529
55809,3B,J. McNeil,NYM,4.9,dk-mlb-20190409-MLB $7.5K Quarter Jukebox [Jus...,2019-04-09 19:05:00,mlb,71238529
55810,SS,D. Swanson,ATL,7.3,dk-mlb-20190409-MLB $7.5K Quarter Jukebox [Jus...,2019-04-09 19:05:00,mlb,71238529
55811,OF,S. Piscotty,OAK,9.2,dk-mlb-20190409-MLB $7.5K Quarter Jukebox [Jus...,2019-04-09 19:05:00,mlb,71238529


,position,name,team_abbr,contest_id
2272,CPT,T. Gonsolin,LAD,94886275
2273,NaN,T. Gonsolin,LAD,94886275
2274,CPT,B. Snell,TB,94886275
2275,NaN,B. Snell,TB,94886275
2276,CPT,J. Turner,LAD,94886275
...,...,...,...,...
55808,2B,T. La Stella,LAA,71238529
55809,3B,J. McNeil,NYM,71238529
55810,SS,D. Swanson,ATL,71238529
55811,OF,S. Piscotty,OAK,71238529


In [39]:
from fantasy_py import FANTASY_SERVICE_DOMAIN, lineup, util

service_cls = util.CLSRegistry.get_class(FANTASY_SERVICE_DOMAIN, SERVICE)
abbr_remaps = service_cls.get_team_abbr_remapping(SPORT)

def fix_team_abbr(abbr) -> str:
    """ make team abbreviations consistent with DB """
    assert isinstance(abbr, str), "expected all players to have a team abbr!"
    return abbr_remaps.get(abbr) or abbr

# add team/lineup draft data
team_contest_df = pd.merge(contest_df, draft_df, on='contest_id')

team_contest_df.team_abbr = team_contest_df.team_abbr.map(fix_team_abbr)
print(f"{len(team_contest_df.contest_id.unique())} contests")
display(team_contest_df)

137 contests


,contest_id,date,title,top_score,last_winning_score,style,type,position,name,team_abbr
0,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP,P,F. Valdez,HOU
1,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP,P,B. Snell,TB
2,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP,C,T. d'Arnaud,ATL
3,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP,1B,F. Freeman,ATL
4,94461340,2020-10-16,MLB $1K Quarter Jukebox [Just $0.25!],168.05,134.8,classic,GPP,2B,M. Brosseau,TB
...,...,...,...,...,...,...,...,...,...,...
15515,71238529,2019-04-09,MLB $7.5K Quarter Jukebox [Just $0.25!] (1/2),225.75,152.8,classic,GPP,2B,T. La Stella,LAA
15516,71238529,2019-04-09,MLB $7.5K Quarter Jukebox [Just $0.25!] (1/2),225.75,152.8,classic,GPP,3B,J. McNeil,NYM
15517,71238529,2019-04-09,MLB $7.5K Quarter Jukebox [Just $0.25!] (1/2),225.75,152.8,classic,GPP,SS,D. Swanson,ATL
15518,71238529,2019-04-09,MLB $7.5K Quarter Jukebox [Just $0.25!] (1/2),225.75,152.8,classic,GPP,OF,S. Piscotty,OAK


In [40]:
import os

# group contests together and create team sets used in each contest
def common_title(titles):
    return os.path.commonprefix(titles.tolist())

teams_contest_df = pd.DataFrame(
    team_contest_df.groupby(
        ['contest_id', 'date', 'style', 'type']
    ).agg(
        {'team_abbr': set,
         'title': common_title,
         'top_score': lambda score: score.mean(),
         'last_winning_score': lambda score: score.mean()}
    )
).reset_index()
teams_contest_df = teams_contest_df.rename(columns={'team_abbr': 'teams'})
teams_contest_df['draft_team_count'] = teams_contest_df.teams.map(len)

display(f"{len(teams_contest_df)} team sets")
display(teams_contest_df)

'137 team sets'

,contest_id,date,style,type,teams,title,top_score,last_winning_score,draft_team_count
0,71238529,2019-04-09,classic,GPP,"{NYY, MIN, WAS, ARI, TEX, LAA, LAD, SEA, SF, H...",MLB $7.5K Quarter Jukebox [Just $0.25!] (,225.75,151.95,18
1,71277566,2019-04-10,classic,GPP,"{PIT, MIN, WAS, ARI, TEX, LAA, SEA, HOU, PHI, ...",MLB $5K Quarter Jukebox [Just $0.25!] (,237.95,192.60,14
2,71360839,2019-04-11,classic,GPP,"{TOR, PIT, ARI, COL, SF, CHC, BOS, SD, ATL, NYM}",MLB $100K Player Appreciation Contest,184.95,163.10,10
3,71364939,2019-04-11,classic,GPP,"{TOR, PIT, ARI, COL, SF, CHC, SD, BOS, ATL, NYM}",MLB $5K Quarter Jukebox [Just $0.25!] (,185.95,142.10,10
4,71369193,2019-04-12,classic,GPP,"{NYY, BAL, ATL, CLE, HOU, KC, OAK, ARI, LAD, P...",MLB $6K Quarter Jukebox [Just $0.25!] (,221.35,176.15,20
...,...,...,...,...,...,...,...,...,...
132,91001160,2020-09-04,classic,GPP,"{NYM, ARI, LAD, SF, CHC, SD, HOU, PHI, SEA, KC...",MLB $4K Quarter Jukebox [Just $0.25!] (,201.45,177.45,11
133,91275476,2020-09-08,classic,GPP,"{ARI, LAD, COL, SF, SD, SEA}",MLB $1K Quarter Jukebox [Just $0.25!] (Night) (,189.30,160.25,6
134,91577995,2020-09-10,classic,GPP,"{ARI, SF, SD, LAD}",MLB $500 Quarter Jukebox [Just $0.25!] (Night) (,115.95,89.95,4
135,91766081,2020-09-12,classic,GPP,"{KC, MIN, PIT, ARI, CIN, CLE, LAA, COL, LAD, C...",MLB $300 Dime Time [Just $0.10!] (,200.30,149.70,14


In [41]:
# load slate data from db
import sqlite3
import pandas as pd

conn = sqlite3.connect(DB_FILENAME)
sql = f"""
select distinct daily_fantasy_slate.id as slate_id, date, 
    daily_fantasy_slate.name as slate_name, style as contest_style, abbr
from daily_fantasy_slate 
    join daily_fantasy_cost on daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
    join team on team_id = team.id
where service = '{SERVICE}' and date between '{MIN_DATE}' and '{MAX_DATE}'
"""

if STYLE is not None:
    sql += f" and style = '{STYLE.name}'"

# print(sql)
db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
# with pd.option_context('max_rows', 100):
#     display(db_df)
conn.close()

# get team sets
slate_db_df = pd.DataFrame(
    db_df.groupby(
        ['slate_id', 'date', 'slate_name', 'contest_style']
    ).agg(
        {'abbr': set}
    )
).reset_index()
slate_db_df = slate_db_df.set_index('date').rename(columns={'abbr': 'teams'})
slate_db_df['team_count'] = slate_db_df.teams.map(len)
with pd.option_context('max_rows', 100):
    display(slate_db_df)

,slate_id,slate_name,contest_style,teams,team_count
date,,,,,
2019-03-20,1983,Unnamed-CLASSIC-Slate-25456,CLASSIC,"{CHW, PIT, MIN, ARI, CIN, TB, CLE, LAA, COL, L...",18
2019-03-21,1986,Unnamed-CLASSIC-Slate-25456,CLASSIC,"{CHW, PIT, MIN, ARI, CIN, TB, CLE, LAA, COL, L...",18
2019-03-28,1987,Unnamed-CLASSIC-Slate-25456,CLASSIC,"{CHW, PIT, MIN, ARI, CIN, TB, CLE, LAA, COL, L...",18
2019-03-28,1988,(Early),CLASSIC,"{NYY, TOR, WAS, DET, BAL, COL, PHI, SF, ATL, N...",12
2019-03-28,1990,(All Day),CLASSIC,"{NYY, BAL, CLE, HOU, SF, STL, KC, OAK, ARI, CI...",28
...,...,...,...,...,...
2020-10-13,4592,Unnamed-CLASSIC-Slate-40698,CLASSIC,"{TB, HOU, LAD, ATL}",4
2020-10-14,4595,Unnamed-CLASSIC-Slate-40779,CLASSIC,"{TB, HOU, LAD, ATL}",4
2020-10-15,4598,Unnamed-CLASSIC-Slate-40850,CLASSIC,"{TB, HOU, LAD, ATL}",4


In [42]:
import numpy as np
from typing import Optional


def get_slate_id(contest_row) -> Optional[pd.Series]:
    """ 
    guesses the db slate id contest_row
    returns - series of (slate_id, number of teams playing in slate)
    """
    try:
        date_slates = slate_db_df.loc[[contest_row.date]].sort_values('team_count')
    except KeyError as ke:
        print(f"Key error finding slates for {contest_row.date}")
        return None
    try:
        slates = date_slates.query("@contest_row.teams <= teams")
    except Exception as e:
        print(f"Unhandled exception querying for teams date {contest_row.date}")
        # display(date_slates)
        raise
        
    slates_found = len(slates)
    if slates_found == 0:
        print(f"On {contest_row.date} the {len(date_slates)} db slates don't match contest teams {contest_row.teams}. "
              "DB slate team sets were:")
        with pd.option_context('max_colwidth', None):
            display(date_slates[['slate_name', 'teams']])
        return None
    if slates_found > 1:
        # display(slates)
        slates = slates.head(1)
        print(f"{slates_found} slates matched contest {contest_row.date} '{contest_row.title}'. "
              f"Using '{slates.iloc[0].slate_name}'")
    return slates.iloc[0][['slate_id', 'team_count']]
    
slate_ids_df = teams_contest_df.apply(get_slate_id, axis=1)
display(slate_ids_df)

2 slates matched contest 2019-04-09 00:00:00 'MLB $7.5K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26133'
2 slates matched contest 2019-04-10 00:00:00 'MLB $5K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26158'
2 slates matched contest 2019-04-11 00:00:00 'MLB $100K Player Appreciation Contest'. Using 'Unnamed-CLASSIC-Slate-26205'
2 slates matched contest 2019-04-11 00:00:00 'MLB $5K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26205'
2 slates matched contest 2019-04-13 00:00:00 'MLB $4K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26250'
2 slates matched contest 2019-04-17 00:00:00 'MLB $5K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26357'
2 slates matched contest 2019-04-18 00:00:00 'MLB $5K Quarter Jukebox [Just $0.25!]'. Using 'Unnamed-CLASSIC-Slate-26387'
2 slates matched contest 2019-04-20 00:00:00 'MLB $5K Quarter Jukebox [Just $0.25!] ('. Using 'Unnamed-CLASSIC-Slate-26479'
2 slates m

,slate_name,teams
date,,
2019-08-09,(Turbo)-2,"{PIT, MIN, CLE, STL, TEX, MIL}"
2019-08-09,Unnamed-CLASSIC-Slate-28855,"{ARI, TB, LAD, COL, SF, SD, PHI, SEA}"
2019-08-09,(Turbo),"{NYY, TOR, WAS, DET, CIN, BAL, ATL, LAA, CHC, HOU, BOS, MIA, NYM, KC}"
2019-08-09,(Happy Hour),"{NYY, TOR, KC, WAS, DET, CIN, BAL, ATL, LAA, CHC, HOU, BOS, MIA, NYM, CHW, OAK}"


2 slates matched contest 2020-09-08 00:00:00 'MLB $1K Quarter Jukebox [Just $0.25!]  (Night) ('. Using 'Unnamed-CLASSIC-Slate-39402'
2 slates matched contest 2020-09-10 00:00:00 'MLB $500 Quarter Jukebox [Just $0.25!] (Night) ('. Using '(Night)'


,slate_id,team_count
0,2095.0,20.0
1,2111.0,18.0
2,2118.0,10.0
3,2118.0,10.0
4,2129.0,24.0
...,...,...
132,4373.0,18.0
133,4410.0,6.0
134,4426.0,4.0
135,4435.0,16.0


In [43]:
# slate game score info
conn = sqlite3.connect(DB_FILENAME)

# for mlb double headers this will cause inaccuracy for players that played in both games
slate_ids_str = ','.join(map(str, slate_ids_df.slate_id.dropna()))
sql = f"""
select distinct daily_fantasy_slate.id as slate_id, game.id as game_id, game.score_home, game.score_away
from daily_fantasy_slate
    join daily_fantasy_cost on daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
    join game on (game.date = daily_fantasy_slate.date and
                  game.season = daily_fantasy_slate.season and 
                  (daily_fantasy_cost.team_id in (game.away_team_id, game.home_team_id)))
where daily_fantasy_slate.id in ({slate_ids_str})
"""

# print(sql)
db_team_score_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
conn.close()
# display(db_team_score_df)
team_score_df = db_team_score_df.melt(id_vars=['slate_id', 'game_id'], value_vars=['score_home', 'score_away']) \
          .groupby(['slate_id']) \
          .agg({'value': ['median', lambda x: np.percentile(x, 75)]})
team_score_df.columns = ['team-med', 'team-75th_pctl']
display(team_score_df)

,team-med,team-75th_pctl
slate_id,,
2095,6.0,8.00
2111,5.0,6.75
2118,4.5,6.00
2129,6.0,8.00
2139,3.5,5.00
...,...,...
4373,5.0,6.00
4410,7.5,9.75
4426,3.5,5.25


In [44]:
# get position scores
conn = sqlite3.connect(DB_FILENAME)

stat_names = (
    f"'{SERVICE_ABBR}_score'"
    if SPORT != 'nfl' else
    f"'{SERVICE_ABBR}_score_off','{SERVICE_ABBR}_score_def'"
)

# for mlb double headers this will cause inaccuracy for players that played in both games
sql = f"""
select daily_fantasy_slate.id as slate_id, positions as position, value as score, 
    daily_fantasy_cost.team_id, daily_fantasy_cost.player_id
from daily_fantasy_slate
    join daily_fantasy_cost on daily_fantasy_slate.id = daily_fantasy_cost.daily_fantasy_slate_id
    join game on (game.date = daily_fantasy_slate.date and
                  game.season = daily_fantasy_slate.season and 
                  (daily_fantasy_cost.team_id in (game.away_team_id, game.home_team_id)))
    join calculation_datum on (calculation_datum.game_id = game.id and 
                               calculation_datum.player_id is daily_fantasy_cost.player_id and
                               calculation_datum.team_id = daily_fantasy_cost.team_id)
    join statistic on calculation_datum.statistic_id = statistic.id
where daily_fantasy_slate.id in ({slate_ids_str}) and
    statistic.name in ({stat_names})
"""
# print(sql)

db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
conn.close()
db_exploded_pos_df = db_df.assign(position=db_df.position.str.split('/')) \
             .explode('position') \
             .query('position not in @POS_DROP')
for old_pos, new_pos in COST_POS_RENAME.items():
    db_exploded_pos_df.loc[db_exploded_pos_df.position == old_pos, 'position'] = new_pos
display(db_exploded_pos_df)

db_pos_scores_df = db_exploded_pos_df[['slate_id', 'position', 'score']] \
             .groupby(['slate_id', 'position']) \
             .agg(['median', lambda x: np.percentile(x, 75)]) 
db_pos_scores_df.columns = ['med-dfs', '75th-pctl-dfs']
db_pos_scores_df = db_pos_scores_df.reset_index(level='position') \
             .pivot(columns='position', values=['med-dfs', '75th-pctl-dfs'])
display(db_pos_scores_df)

,slate_id,position,score,team_id,player_id
0,2095,P,18.207,1,1330
1,2095,1B,35.000,1,2226
1,2095,2B,35.000,1,2226
2,2095,OF,5.000,1,2260
3,2095,C,12.000,1,2383
...,...,...,...,...,...
22845,4435,2B,5.000,30,3353
22846,4435,OF,3.000,30,3462
22847,4435,2B,0.000,30,3610
22847,4435,OF,0.000,30,3610


med-dfs                                      75th-pctl-dfs         \
position      1B   2B    3B     C   OF        P    SS            1B     2B   
slate_id                                                                     
2095         5.0  6.5   4.0   6.0  5.0  10.1290   6.0          9.75  13.25   
2111         4.0  3.0   4.0   2.0  5.0  15.1000   6.0          9.25  12.00   
2118         5.0  3.0   2.0   2.0  3.5  18.0000   6.0          9.50   6.00   
2129         5.0  3.0   7.0   4.5  6.5   9.6500   5.0         10.50  10.00   
2139         5.5  3.0   5.0   2.5  3.5  13.9000   5.0          9.00   8.00   
...          ...  ...   ...   ...  ...      ...   ...           ...    ...   
4359         3.0  3.0   3.0   2.5  3.5  16.6420   2.0          8.50   5.00   
4373         6.0  3.0   5.5   2.0  5.0  13.0755   6.5          9.25   8.00   
4410         9.0  3.5  10.0   6.0  9.0   3.6040  10.5         13.00   9.25   
4426         3.0  9.0   0.0  10.5  5.0   3.8000   6.5         11.50  13.00   
4435         5.0  4.5   3.0   2.0  3.0  13.3000   4.0         14.00   9.50   

                                                
position     3B      C     OF         P     SS  
slate_id                                        
2095       7.00  12.00  11.00  17.71425  14.00  
2111      12.00   7.00  12.00  27.45000   8.00  
2118       7.00   8.00   7.75  25.53550   9.50  
2129      11.50   8.25  12.00  15.92150  12.00  
2139      11.00   9.50   8.75  20.60000   8.00  
...         ...    ...    ...       ...    ...  
4359       5.75   5.50   8.50  22.70000   5.75  
4373       8.50   5.50  10.00  23.45000  12.75  
4410      15.00  11.50  14.00   3.74600  13.75  
4426       3.00  14.00   7.00   6.86250  11.00  
4435      11.25   6.50  11.00  18.15000  11.00  

[109 rows x 14 columns]

In [48]:
# Get position score for top players (e.g. players that are likely to be highly drafted)
conn = sqlite3.connect(DB_FILENAME)

sql = f"""
select game.date, calculation_datum.player_id, calculation_datum.team_id, calculation_datum.value as score 
from game
    join calculation_datum on calculation_datum.game_id = game.id
    join statistic on calculation_datum.statistic_id = statistic.id
where statistic.name in ({stat_names}) 
    and date between date('{MIN_DATE}', '-{TOP_PLAYER_DAYS} days') and '{MAX_DATE}'
"""
print(sql)
db_df = pd.read_sql_query(sql, conn, parse_dates=['date'])
conn.close()
# display(db_df)

db_filtered_df = db_df.query(
    '(player_id in @db_exploded_pos_df.player_id) '
    'or (player_id.isnull() and team_id in @db_exploded_pos_df.team_id)'
)
display(db_filtered_df)


select game.date, calculation_datum.player_id, calculation_datum.team_id, calculation_datum.value as score 
from game
    join calculation_datum on calculation_datum.game_id = game.id
    join statistic on calculation_datum.statistic_id = statistic.id
where statistic.name in ('dk_score') 
    and date between date('2019-01-01', '-21 days') and '2021-01-01'



,date,player_id,team_id,score
0,2019-04-09,1,2,3.0
1,2019-04-11,1,2,5.0
2,2019-04-13,1,2,4.0
3,2019-04-14,1,2,0.0
4,2019-04-15,1,2,0.0
...,...,...,...,...
67545,2020-09-22,4165,11,7.0
67546,2020-09-25,4165,11,2.0
67547,2020-09-25,4165,11,16.0
67548,2020-09-26,4165,11,2.0


In [46]:
predict_df = pd.concat(
    [teams_contest_df[['date', 'style', 'type', 'top_score', 'last_winning_score']],
     slate_ids_df],
    axis='columns',
).join(team_score_df, on='slate_id') \
 .join(db_pos_scores_df, on='slate_id')

display(predict_df)

/home/delano/working/fantasy/venv-3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,date,style,type,top_score,last_winning_score,slate_id,team_count,team-med,team-75th_pctl,"(med-dfs, 1B)",...,"(med-dfs, OF)","(med-dfs, P)","(med-dfs, SS)","(75th-pctl-dfs, 1B)","(75th-pctl-dfs, 2B)","(75th-pctl-dfs, 3B)","(75th-pctl-dfs, C)","(75th-pctl-dfs, OF)","(75th-pctl-dfs, P)","(75th-pctl-dfs, SS)"
0,2019-04-09,classic,GPP,225.75,151.95,2095.0,20.0,6.0,8.00,5.0,...,5.0,10.1290,6.0,9.75,13.25,7.00,12.00,11.00,17.71425,14.00
1,2019-04-10,classic,GPP,237.95,192.60,2111.0,18.0,5.0,6.75,4.0,...,5.0,15.1000,6.0,9.25,12.00,12.00,7.00,12.00,27.45000,8.00
2,2019-04-11,classic,GPP,184.95,163.10,2118.0,10.0,4.5,6.00,5.0,...,3.5,18.0000,6.0,9.50,6.00,7.00,8.00,7.75,25.53550,9.50
3,2019-04-11,classic,GPP,185.95,142.10,2118.0,10.0,4.5,6.00,5.0,...,3.5,18.0000,6.0,9.50,6.00,7.00,8.00,7.75,25.53550,9.50
4,2019-04-12,classic,GPP,221.35,176.15,2129.0,24.0,6.0,8.00,5.0,...,6.5,9.6500,5.0,10.50,10.00,11.50,8.25,12.00,15.92150,12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2020-09-04,classic,GPP,201.45,177.45,4373.0,18.0,5.0,6.00,6.0,...,5.0,13.0755,6.5,9.25,8.00,8.50,5.50,10.00,23.45000,12.75
133,2020-09-08,classic,GPP,189.30,160.25,4410.0,6.0,7.5,9.75,9.0,...,9.0,3.6040,10.5,13.00,9.25,15.00,11.50,14.00,3.74600,13.75
134,2020-09-10,classic,GPP,115.95,89.95,4426.0,4.0,3.5,5.25,3.0,...,5.0,3.8000,6.5,11.50,13.00,3.00,14.00,7.00,6.86250,11.00
135,2020-09-12,classic,GPP,200.30,149.70,4435.0,16.0,4.5,7.00,5.0,...,3.0,13.3000,4.0,14.00,9.50,11.25,6.50,11.00,18.15000,11.00
